In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
        

## 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

In [9]:
filepath="../data/multi_asset_etf_data.xlsx"
df_excessreturn = pd.read_excel(
    filepath, sheet_name="excess returns", index_col="Date")

df_summary = pd.DataFrame(index=df_excessreturn.columns)
df_summary["mean"] = df_excessreturn.mean(axis=0) * 12
df_summary["std"] = df_excessreturn.std(axis=0) * 12**0.5
df_summary=df_summary.assign(sharpe_ratio=lambda x: x["mean"] / x["std"])
df_summary

,mean,std,sharpe_ratio
BWX,-0.001843,0.083359,-0.022112
DBC,0.025443,0.178975,0.142162
EEM,0.064887,0.196531,0.330163
EFA,0.081597,0.165991,0.491573
HYG,0.064168,0.089154,0.719746
IEF,0.014269,0.062405,0.228652
IYR,0.129473,0.187101,0.691997
PSP,0.079938,0.227387,0.351552
QAI,0.018974,0.050810,0.373440
SPY,0.143727,0.147679,0.973245


In [10]:
print(f" The asset has the worst Sharpe ratios is {df_summary['sharpe_ratio'].idxmin()} ({df_summary['sharpe_ratio'].min()})")
print(f" The asset have the best Sharpe ratios is {df_summary['sharpe_ratio'].idxmax()}({df_summary['sharpe_ratio'].max()})")

 The asset has the worst Sharpe ratios is BWX (-0.022112401285017045)
 The asset have the best Sharpe ratios is SPY(0.9732449799937645)


## 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

In [11]:
corr=df_excessreturn.corr()
corr

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.000000,0.349773,0.647614,0.621662,0.557653,0.434472,0.453534,0.524870,0.668045,0.465713,0.617099
DBC,0.349773,1.000000,0.565654,0.581865,0.473208,-0.321738,0.318314,0.496057,0.547936,0.509886,0.136668
EEM,0.647614,0.565654,1.000000,0.851579,0.726041,-0.102347,0.621814,0.771677,0.807245,0.734556,0.302729
EFA,0.621662,0.581865,0.851579,1.000000,0.771463,-0.132331,0.697875,0.891929,0.853674,0.871641,0.287476
HYG,0.557653,0.473208,0.726041,0.771463,1.000000,-0.008598,0.757649,0.823823,0.768756,0.770353,0.365939
IEF,0.434472,-0.321738,-0.102347,-0.132331,-0.008598,1.000000,0.073622,-0.118676,0.055667,-0.155696,0.706078
IYR,0.453534,0.318314,0.621814,0.697875,0.757649,0.073622,1.000000,0.760158,0.655963,0.753610,0.397166
PSP,0.524870,0.496057,0.771677,0.891929,0.823823,-0.118676,0.760158,1.000000,0.838287,0.895729,0.320913
QAI,0.668045,0.547936,0.807245,0.853674,0.768756,0.055667,0.655963,0.838287,1.000000,0.840989,0.459712
SPY,0.465713,0.509886,0.734556,0.871641,0.770353,-0.155696,0.753610,0.895729,0.840989,1.000000,0.294639


### SPY and PSP has the highest correlation(0.895729), and IEF and DBC has the lowest correlation(-0.321738)

### Compared with domestic bonds which are represented by IEF(7-10Y Treasury Bond), HYG(Corp Bond), TIPS outperformed IEF(7-10Y Treasury Bond) with a relatively higher expected return and lower volatility , but cannot outperfrom high yield Corp Bond, which have higher return and sharpe ratio

### Compared with foreign bonds(PSP, BWX, EFA, EEM), TIPS have a relatively same excess return but a obviously higher sharpe ratio than foreign bonds.

## 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

By using the formula $w^{tan}$=$\frac{1}{I'\Sigma^{-1}\mu}$$\Sigma^{-1}\mu$:

In [12]:
def pfl_Mv(df_excessreturn,df_summary):
    sigma=df_excessreturn.cov()*12
    sigma=np.mat(sigma.values)
    
    mu=np.mat(df_summary['mean'].values)
    I=np.ones([1,len(sigma)])
    
    w_tan=sigma.getI()*mu.T/(
        I*sigma.getI()*mu.T
    )
    df_w=pd.DataFrame(w_tan,index=df_summary.index,columns=['Weight'])
    df_w['sharpe_ratio']=df_summary['sharpe_ratio']
    df_w.sort_values('Weight',ascending=False)
    
    mu_pfl=(mu*w_tan)[0,0]
    Vol_pfl=(w_tan.T*sigma*w_tan)[0,0]
    W_tan_Sharpe=mu_pfl/Vol_pfl**0.5
    
    return w_tan,df_w,mu_pfl,Vol_pfl,W_tan_Sharpe

w_tan,df_w,mu_pfl,Vol_pfl,W_tan_Sharpe=pfl_Mv(df_excessreturn,df_summary)
df_w

,Weight,sharpe_ratio
BWX,-1.464974,-0.022112
DBC,0.028436,0.142162
EEM,0.261028,0.330163
EFA,0.452914,0.491573
HYG,1.528942,0.719746
IEF,1.893992,0.228652
IYR,-0.242772,0.691997
PSP,-1.271055,0.351552
QAI,-3.133445,0.373440
SPY,2.589999,0.973245


In [13]:
mu_pfl,Vol_pfl,W_tan_Sharpe

(0.37018005312751834, 0.036681013041624515, 1.9328241918711755)

### Asset with higher sharpe ratio tend to have higher weight, but it is not always true because the correlation between different assets are considered when optimizing the portfolio.


### The tangency portfolio corresponding to $w^{tan}$ has a return of 0.3701, a variance of 0.0366, a Sharpe ratio of 1.9328

## 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

### drop TIPS:

In [14]:
df_excessreturn = pd.read_excel(
    filepath, sheet_name="excess returns", index_col="Date").drop(columns=['TIP'])

df_summary = pd.DataFrame(index=df_excessreturn.columns)
df_summary["mean"] = df_excessreturn.mean(axis=0) * 12
df_summary["std"] = df_excessreturn.std(axis=0) * 12**0.5
df_summary=df_summary.assign(sharpe_ratio=lambda x: x["mean"] / x["std"])
w_tan,df_w,mu_pfl,Vol_pfl,W_tan_Sharpe=pfl_Mv(df_excessreturn,df_summary)
df_w

,Weight,sharpe_ratio
BWX,-1.512750,-0.022112
DBC,0.055158,0.142162
EEM,0.278086,0.330163
EFA,0.441496,0.491573
HYG,1.593140,0.719746
IEF,2.212451,0.228652
IYR,-0.245895,0.691997
PSP,-1.314092,0.351552
QAI,-3.238956,0.373440
SPY,2.731363,0.973245


In [15]:
mu_pfl,Vol_pfl,W_tan_Sharpe

(0.3862908162901107, 0.04004440806921932, 1.9303828215222578)

### Adjust expected excess return to TIPS:

In [16]:
df_excessreturn = pd.read_excel(
    filepath, sheet_name="excess returns", index_col="Date")

df_summary = pd.DataFrame(index=df_excessreturn.columns)
df_summary["mean"] = df_excessreturn.mean(axis=0) * 12

df_summary.loc['TIP','mean']=df_summary.loc['TIP','mean']+0.0012

df_summary["std"] = df_excessreturn.std(axis=0) * 12**0.5
df_summary=df_summary.assign(sharpe_ratio=lambda x: x["mean"] / x["std"])
w_tan,df_w,mu_pfl,Vol_pfl,W_tan_Sharpe=pfl_Mv(df_excessreturn,df_summary)
df_w

,Weight,sharpe_ratio
BWX,-1.444459,-0.022112
DBC,0.016962,0.142162
EEM,0.253703,0.330163
EFA,0.457817,0.491573
HYG,1.501376,0.719746
IEF,1.757244,0.228652
IYR,-0.241431,0.691997
PSP,-1.252574,0.351552
QAI,-3.088139,0.373440
SPY,2.529296,0.973245


In [17]:
mu_pfl,Vol_pfl,W_tan_Sharpe

(0.3638742692096192, 0.035341454744001216, 1.9355711271437428)

### After dropping the TIPS, the portfolio achieve both higher return and volatility but decrease slightly -0.0025 in sharpe ratio. This indicates that the TIPS seem to expand the investment opportunity set though not very much.

### After increase the expected return of TIPS by 0.0012, the portfolio achieve both lower return and volatility but increase 0.0027 in sharpe ratio.This indicates that if the TIPS outperform its history, it seem to help provide less volatility and more security when inflation occurs.

# 3. Allocations

* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\mutarg$ = 0.01.

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$






In [18]:
# calculate on monthly data

df_excessreturn = pd.read_excel(
    filepath, sheet_name="excess returns", index_col="Date")

mu = df_excessreturn.mean(axis=0).values
mu=np.mat(mu)
sigma=df_excessreturn.cov().values
sigma=np.mat(sigma)

Equal_weight=np.ones([11,1])*1/11

mu_pfl=(mu*Equal_weight)[0,0]
Vol_pfl=(Equal_weight.T*sigma*Equal_weight)[0,0]
W_tan_Sharpe=mu_pfl/Vol_pfl**0.5
mu_pfl,W_tan_Sharpe,W_tan_Sharpe

# rescale the weight
desired_mu=0.01
Equal_weight=Equal_weight/mu_pfl*desired_mu

mu_pfl=(mu*Equal_weight)[0,0]
Vol_pfl=(Equal_weight.T*sigma*Equal_weight)[0,0]**0.5
W_tan_Sharpe=mu_pfl/Vol_pfl
f"expected return:{mu_pfl:.5f}, volitility:{Vol_pfl:.5f}, sharpe ratio: {W_tan_Sharpe:.5f}"

'expected return:0.01000, volitility:0.06157, sharpe ratio: 0.16242'

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$


In [19]:
RP_weight=np.mat([1/i**2 for i in list(df_excessreturn.std(axis=0))]).T

mu_pfl=(mu*RP_weight)[0,0]
Vol_pfl=(RP_weight.T*sigma*RP_weight)[0,0]
W_tan_Sharpe=mu_pfl/Vol_pfl**0.5

# rescale the weight
desired_mu=0.01
RP_weight=RP_weight/mu_pfl*desired_mu

mu_pfl=(mu*RP_weight)[0,0]
Vol_pfl=(RP_weight.T*sigma*RP_weight)[0,0]**0.5
W_tan_Sharpe=mu_pfl/Vol_pfl
f"expected return:{mu_pfl:.5f}, volitility:{Vol_pfl:.5f}, sharpe ratio: {W_tan_Sharpe:.5f}"

'expected return:0.01000, volitility:0.06284, sharpe ratio: 0.15913'

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 


In [20]:
for i in range(sigma.shape[0]):
    sigma[i,i]+=sigma[i,i]
    
sigma=sigma/2
    
REG_weight=sigma.getI()*mu.T

mu_pfl=(mu*REG_weight)[0,0]
Vol_pfl=(REG_weight.T*sigma*REG_weight)[0,0]
W_tan_Sharpe=mu_pfl/Vol_pfl**0.5

# rescale the weight
desired_mu=0.01
REG_weight=REG_weight/mu_pfl*desired_mu

mu_pfl=(mu*REG_weight)[0,0]
Vol_pfl=(REG_weight.T*sigma*REG_weight)[0,0]**0.5
W_tan_Sharpe=mu_pfl/Vol_pfl
f"expected return:{mu_pfl:.5f}, volitility:{Vol_pfl:.5f}, sharpe ratio: {W_tan_Sharpe:.5f}"    

'expected return:0.01000, volitility:0.02789, sharpe ratio: 0.35849'

### Comparing

In order to compare all these allocation methods, (those above, along with the tangency portfolio obtained in the previous section,) rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How do these compare across the four allocation methods?

### Have Already rescaled in the above code and results. The rescaled result of MV method is shown below:

In [212]:
Mv_weight=sigma.getI()*mu.T

mu_pfl=(mu*Mv_weight)[0,0]
Vol_pfl=(Mv_weight.T*sigma*Mv_weight)[0,0]
W_tan_Sharpe=mu_pfl/Vol_pfl**0.5

# rescale the weight
desired_mu=0.01
Mv_weight=Mv_weight/mu_pfl*desired_mu

mu_pfl=(mu*Mv_weight)[0,0]
Vol_pfl=(Mv_weight.T*sigma*Mv_weight)[0,0]
W_tan_Sharpe=mu_pfl/Vol_pfl**0.5
f"expected return:{mu_pfl:.5f}, volitility:{Vol_pfl:.5f}, sharpe ratio: {W_tan_Sharpe:.5f}"   

'expected return:0.01000, volitility:0.00032, sharpe ratio: 0.55796'

### While achieving the same expected return, (">" means outperforming ) MV > REG > RP > EW with lower volitility and higher sharpe ratio